https://github.com/BAMresearch/NFDI4IngScientificWorkflowRequirements

In [1]:
!conda env list


# conda environments:
#
base                 * /home/janssen/mambaforge
postprocessing         /home/janssen/mambaforge/envs/postprocessing
preprocessing          /home/janssen/mambaforge/envs/preprocessing
processing             /home/janssen/mambaforge/envs/processing



# Preprocessing 

In [2]:
import os

In [3]:
domain_size = 2.0

## generate mesh

In [4]:
from nfdi_ing_workflow import generate_mesh

In [5]:
gmsh_output_file = generate_mesh(
    gmsh_input_file=os.path.abspath("source/unit_square.geo"), 
    domain_size=domain_size,
)
gmsh_output_file

'/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd/preprocessing/square.msh'

In [6]:
!cat {gmsh_output_file}

$MeshFormat
4.1 0 8
$EndMeshFormat
$Entities
4 4 1 0
1 0 0 0 0 
2 2 0 0 0 
3 2 2 0 0 
4 0 2 0 0 
1 -9.999999994736442e-08 -1e-07 -1e-07 2.0000001 1e-07 1e-07 0 2 1 -2 
2 1.9999999 -9.999999994736442e-08 -1e-07 2.0000001 2.0000001 1e-07 0 2 2 -3 
3 -9.999999994736442e-08 1.9999999 -1e-07 2.0000001 2.0000001 1e-07 0 2 3 -4 
4 -1e-07 -9.999999994736442e-08 -1e-07 1e-07 2.0000001 1e-07 0 2 4 -1 
1 -9.999999994736442e-08 -9.999999994736442e-08 -1e-07 2.0000001 2.0000001 1e-07 1 1 4 1 2 3 4 
$EndEntities
$Nodes
9 98 1 98
0 1 0 1
1
0 0 0
0 2 0 1
2
2 0 0
0 3 0 1
3
2 2 0
0 4 0 1
4
0 2 0
1 1 0 7
5
6
7
8
9
10
11
0.2500000000000001 0 0
0.5000000000000001 0 0
0.75 0 0
1 0 0
1.25 0 0
1.5 0 0
1.75 0 0
1 2 0 7
12
13
14
15
16
17
18
2 0.2500000000000001 0
2 0.5000000000000001 0
2 0.75 0
2 1 0
2 1.25 0
2 1.5 0
2 1.75 0
1 3 0 7
19
20
21
22
23
24
25
1.75 2 0
1.5 2 0
1.25 2 0
1 2 0
0.75 2 0
0.5 2 0
0.25 2 0
1 4 0 7
26
27
28
29
30
31
32
0 1.75 0
0 1.5 0
0 1.25 0
0 1 0
0 0.75 0
0 0.5 0
0 0.25 0
2 1 0 66
33
34

## convert to xdmf

In [7]:
from nfdi_ing_workflow import convert_to_xdmf

In [8]:
meshio_output_dict = convert_to_xdmf(gmsh_output_file=gmsh_output_file)
meshio_output_dict

{'xdmf': '/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd/preprocessing/square.xdmf',
 'h5': '/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd/preprocessing/square.h5'}

In [9]:
xdmf_file_name = meshio_output_dict['xdmf']

In [10]:
!cat {xdmf_file_name}

<Xdmf Version="3.0"><Domain><Grid Name="Grid"><Geometry GeometryType="XYZ"><DataItem DataType="Float" Dimensions="98 3" Format="HDF" Precision="8">square.h5:/data0</DataItem></Geometry><Topology TopologyType="Triangle" NumberOfElements="162" NodesPerElement="3"><DataItem DataType="Int" Dimensions="162 3" Format="HDF" Precision="8">square.h5:/data1</DataItem></Topology><Attribute Name="gmsh:dim_tags" AttributeType="Vector" Center="Node"><DataItem DataType="Int" Dimensions="98 2" Format="HDF" Precision="8">square.h5:/data2</DataItem></Attribute><Attribute Name="gmsh:physical" AttributeType="Scalar" Center="Cell"><DataItem DataType="Int" Dimensions="162" Format="HDF" Precision="8">square.h5:/data3</DataItem></Attribute><Attribute Name="gmsh:geometrical" AttributeType="Scalar" Center="Cell"><DataItem DataType="Int" Dimensions="162" Format="HDF" Precision="8">square.h5:/data4</DataItem></Attribute></Grid></Domain></Xdmf>

# Processing

## poisson

In [11]:
from nfdi_ing_workflow import poisson

In [12]:
poisson_dict = poisson(
    meshio_output_xdmf=meshio_output_dict["xdmf"], 
    meshio_output_h5=meshio_output_dict["h5"],
)
poisson_dict

{'numdofs': 357,
 'poisson_output_pvd_file': '/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd/processing/poisson.pvd',
 'poisson_output_vtu_file': '/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd/processing/poisson000000.vtu'}

In [13]:
poisson_file_name = poisson_dict['poisson_output_pvd_file']

In [14]:
!cat {poisson_file_name}

<?xml version="1.0"?>
<VTKFile type="Collection" version="0.1">
  <Collection>
    <DataSet timestep="0" part="0" file="poisson000000.vtu" />
  </Collection>
</VTKFile>


# Postprocessing

## plot over line

In [15]:
from nfdi_ing_workflow import plot_over_line

In [16]:
pvbatch_output_file = plot_over_line(poisson_output_pvd_file=poisson_dict["poisson_output_pvd_file"], poisson_output_vtu_file=poisson_dict['poisson_output_vtu_file'])
pvbatch_output_file

'/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd/postprocessing/plotoverline.csv'

In [17]:
!cat {pvbatch_output_file}

"u","vtkValidPointMask","arc_length","Points:0","Points:1","Points:2"
1,1,0,0,0,0
1.0011,1,0.0028284,0.002,0.002,0
1.0022,1,0.0056569,0.004,0.004,0
1.0033,1,0.0084853,0.006,0.006,0
1.0044,1,0.011314,0.008,0.008,0
1.0055,1,0.014142,0.01,0.01,0
1.0066,1,0.016971,0.012,0.012,0
1.0077,1,0.019799,0.014,0.014,0
1.0088,1,0.022627,0.016,0.016,0
1.0099,1,0.025456,0.018,0.018,0
1.011,1,0.028284,0.02,0.02,0
1.0121,1,0.031113,0.022,0.022,0
1.0132,1,0.033941,0.024,0.024,0
1.0143,1,0.03677,0.026,0.026,0
1.0154,1,0.039598,0.028,0.028,0
1.0165,1,0.042426,0.03,0.03,0
1.0176,1,0.045255,0.032,0.032,0
1.0187,1,0.048083,0.034,0.034,0
1.0198,1,0.050912,0.036,0.036,0
1.0209,1,0.05374,0.038,0.038,0
1.022,1,0.056569,0.04,0.04,0
1.0231,1,0.059397,0.042,0.042,0
1.0242,1,0.062225,0.044,0.044,0
1.0253,1,0.065054,0.046,0.046,0
1.0264,1,0.067882,0.048,0.048,0
1.0275,1,0.070711,0.05,0.05,0
1.0285,1,0.073539,0.052,0.052,0
1.0296,1,0.076368,0.054,0.054,0
1.0307,1,0.079196,0.056,0.056,0
1.0318,1,0.082024,0.058,0.058,0
1

## substitute macros

In [18]:
from nfdi_ing_workflow import substitute_macros

In [19]:
macros_tex_file = substitute_macros( 
    pvbatch_output_file=pvbatch_output_file, 
    ndofs=poisson_dict["numdofs"], 
    domain_size=domain_size,
)
macros_tex_file

'/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd/postprocessing/macros.tex'

## compile paper

In [20]:
from nfdi_ing_workflow import compile_paper

In [21]:
paper_output = compile_paper(macros_tex=macros_tex_file, plot_file=pvbatch_output_file)
paper_output

'/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd/postprocessing/paper.pdf'